<a href="https://colab.research.google.com/github/Max-FM/SPRINT-Colombia/blob/main/Downloading_Satellite_Images_From_Google_Earth_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Downloading Satellite Images From Google Earth Engine 

## Install geemap

[geemap](https://geemap.readthedocs.io/en/latest/readme.html) is a useful package that adds additional functionality to the Google Earth Engine Python API.

In [29]:
%%capture

!pip install geemap

##Import required packages

In [43]:
import ee
import glob
import folium
import os.path
import pandas as pd
import geemap.eefolium as emap

##Authenticate Google Earth Engine

To access the Google Earth Engine API you require an account. To request access, go to [https://signup.earthengine.google.com](https://signup.earthengine.google.com/). You may have to wait up to a day or so to be granted access and it's possible you will not recieve any email communication. To manually check whether you have access, try to log into [https://code.earthengine.google.com](https://code.earthengine.google.com/), or attempt to run the next cell and follow the instructions provided in the output cell.

In [31]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=oBHpeY6tloQRxhK_rB0e4VOOiTGs0H7xmSc8UzRyuuA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/5wFh_JtPEx8tiAJOz1HAR9SOt1vfe25AE34ukSyJ1C8hrrw4l-XQF1A

Successfully saved authorization token.


## Define Request Function

Defines a series of Google Earth Image Collections that we wish to download images from. Image collections are filtered to a input geographical region, date range and maxiumum allowed cloud coverage.

In [32]:
def obtain_data(region, start_date, end_date, months_either_side=0, max_cloud_cover=80):
    start_date = ee.Date(start_date)
    start_date = start_date.advance(-months_either_side, 'month')

    end_date = ee.Date(end_date)
    end_date = end_date.advance(months_either_side, 'month')
    
    # Filter input collections by desired date range, region and cloud coverage.
    criteria  = ee.Filter.And(ee.Filter.geometry(region), 
                              ee.Filter.date(start_date, end_date))
    
    Sentinel_2_SR = ee.ImageCollection('COPERNICUS/S2_SR') \
                      .filter(criteria) \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)) \
                      .select(['B4', 'B3', 'B2', 'B8'])

    Sentinel_1_SAR_GRD_C_BAND = ee.ImageCollection("COPERNICUS/S1_GRD") \
                                  .filter(criteria)

    Landsat_8_T1_SR = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
                        .filter(criteria) \
                        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover)) \
                        .select(['B4', 'B3', 'B2', 'B5'])

    Landsat_7_T1_SR = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") \
                        .filter(criteria) \
                        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover)) \
                        .select(['B3', 'B2', 'B1', 'B4'])

    Landsat_5_T1_SR = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
                        .filter(criteria) \
                        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover)) \
                        .select(['B3', 'B2', 'B1', 'B4'])

    MODIS_16D_NDVI = ee.ImageCollection("MODIS/006/MOD13Q1") \
                       .filter(criteria) \
                       .select('NDVI')

    image_collections = {'Sentinel_2_SR': Sentinel_2_SR,
                         'Sentinel_1_SAR_GRD_C_BAND': Sentinel_1_SAR_GRD_C_BAND,
                         'Landsat_8_T1_SR': Landsat_8_T1_SR,
                         'Landsat_7_T1_SR': Landsat_7_T1_SR,
                         'Landsat_5_T1_SR': Landsat_5_T1_SR,
                         'MODIS_16D_NDVI': MODIS_16D_NDVI}

    return image_collections

## Importing Table of Disaster Dates

In [33]:
disaster_dates = pd.read_csv('/content/drive/Shared drives/Colombia SPRINT/Test Districts/disaster_dates.csv', index_col=0)

# Options are 'Flood', 'Drought', 'Wildfire', 'Earthquake' and 'Storm'. 
# Selects all if 'None'.
chosen_disaster = None

if chosen_disaster:
    disaster_dates = disaster_dates[disaster_dates['Disaster'] == chosen_disaster]

disaster_dates

,Disaster,Start_Datetime,End_Datetime
0,Drought,1998-01-01,1999-01-01
1,Flood,1999-01-10,1999-05-19
2,Earthquake,1999-01-25,1999-01-26
3,Flood,1999-10-28,1999-12-31
4,Flood,2000-05-18,2000-05-24
5,Wildfire,2001-08-01,2001-09-01
6,Drought,2002-01-01,2003-01-01
7,Flood,2002-04-24,2002-04-29
8,Flood,2003-08-01,2003-12-01
9,Flood,2004-01-01,2004-06-28


## Initialising Parameters

In [34]:
# Points to the geometries uploaded to Google Earth Engine for each test district.
test_districts = {'Dosquebradas': ee.FeatureCollection('users/maxfoxley-marrable/SPRINT/Dosquebradas'),
                  'Versailles': ee.FeatureCollection('users/maxfoxley-marrable/SPRINT/Versailles')}

# Defines the desired pixel scale for each image, have set to the native 
# resolution of each satellite.
scale_dict = {'Sentinel_2_SR': 10,
              'Sentinel_1_SAR_GRD_C_BAND': 10,
              'Landsat_8_T1_SR': 30,
              'Landsat_7_T1_SR': 30,
              'Landsat_5_T1_SR': 30,
              'MODIS_16D_NDVI': 250}

# Defines the desired bands, specifically when making a timelapse GIF.
bands_dict = {'Sentinel_2_SR': ['B4', 'B3', 'B2'], 
              'Landsat_8_T1_SR': ['B4', 'B3', 'B2'],
              'Landsat_7_T1_SR': ['B3', 'B2', 'B1'],
              'Landsat_5_T1_SR': ['B3', 'B2', 'B1'],
              'MODIS_16D_NDVI': 'NDVI'} 

##Downloading Imaging for All Events

In [38]:
%%time

# e.g ['Sentinel_2_SR', 'Landsat_8_T1_SR', etc]. Selects all if empty/None.
desired_collections = None

# Set to True to overwrite existing TIFF files.
overwrite = False

# Iterating through each of the test districts.
for district_name, district_geom in test_districts.items():
    display(district_name)
    # Defines base directory to save images to.
    base_dir = f'/content/drive/Shared drives/Colombia SPRINT/Test Districts/{district_name}/Extreme Weather Event Imagery/'
    %cd '{base_dir}'

    # Iterating though each disaster in the table.
    for index, disaster in disaster_dates.iterrows():
        disaster_type = disaster['Disaster']
        start_date = disaster['Start_Datetime']
        end_date = disaster['End_Datetime']

        display(f'{disaster_type}: {start_date} - {end_date}')

        # Creates folder for each disaster type. 
        disaster_dir = f'{disaster_type}s'
        !mkdir '{disaster_dir}'

        # Obtains all image collections defined in the request function for 
        # the chosen test district and date range. Looking 2 months either side
        # of the given date range and with maximum 80% cloud cover.
        image_collections = obtain_data(district_geom, 
                                        start_date, 
                                        end_date,
                                        months_either_side=2,
                                        max_cloud_cover=80)
        
        if disaster_type == 'Drought':
            # Probably only want MODIS 16D imagery for El Nino droughts.
            image_collections = {'MODIS_16D_NDVI': image_collections['MODIS_16D_NDVI']}
        
        # Filters out unwanted collections if defined above.
        elif desired_collections:
            image_collections = {collection: image_collections[collection] \
                                 for collection in desired_collections}    
        
        # Creates a subfolder in the base directory for the start date of the 
        # disaster.
        out_dir = f'{disaster_dir}/{start_date}'
        !mkdir '{out_dir}'

        # Iterating through each image collection.
        for collection_name, collection in image_collections.items():
            # Counts the number of images in a collection.
            collection_size = collection.size().getInfo()
            
            # Skips the image collection if it contains no images.
            if collection_size == 0:
                print('No images in collection, skipping.')
                continue
            # Creates additional subfolders for each image collection. 
            collection_dir = f'{out_dir}/{collection_name}'
            !mkdir '{collection_dir}'

            # Counts number of .tif files already in image collection subfolder.
            tif_count = len(glob.glob1(collection_dir,"*.tif"))

            # Assumes the download for this collection is already complete and 
            # therefore skips, provided the number of .tif files already in 
            # chosen directory matches the number of images in the collection 
            # and overwrite is set to False.
            if collection_size == tif_count and overwrite == False:
                print('Correct number of .tif files for image collection already in directory, skipping.')
                continue

            # Exports each image in the filtered image collection to 
            # geoTIFF format.
            emap.ee_export_image_collection(collection, 
                                            collection_dir,
                                            crs='EPSG:4326',
                                            scale=scale_dict[collection_name],
                                            region=district_geom.geometry().bounds())


'Dosquebradas'

/content/drive/Shared drives/Colombia SPRINT/Test Districts/Dosquebradas/Extreme Weather Event Imagery


'Drought: 1998-01-01 - 1999-01-01'

Droughts
mkdir: cannot create directory ‘Droughts’: File exists
Droughts/1998-01-01
mkdir: cannot create directory ‘Droughts/1998-01-01’: File exists
No images in collection, skipping.


'Flood: 1999-01-10 - 1999-05-19'

...

Droughts
mkdir: cannot create directory ‘Droughts’: File exists
Droughts/2018-01-01
mkdir: cannot create directory ‘Droughts/2018-01-01’: File exists
Droughts/2018-01-01/MODIS_16D_NDVI
mkdir: cannot create directory ‘Droughts/2018-01-01/MODIS_16D_NDVI’: File exists
Total number of images: 54

Exporting 1/54: 2017_11_01.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Droughts/2018-01-01/MODIS_16D_NDVI/2017_11_01.tif


Exporting 2/54: 2017_11_17.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Droughts/2018-01-01/MODIS_16D_NDVI/2017_11_17.tif


Exporting 3/54: 2017_12_03.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Droughts/2018-01-01/MODIS_16D_NDVI/2017_12_0

'Flood: 2019-02-20 - 2019-02-26'

Floods
mkdir: cannot create directory ‘Floods’: File exists
Floods/2019-02-20
mkdir: cannot create directory ‘Floods/2019-02-20’: File exists
Floods/2019-02-20/Sentinel_2_SR
mkdir: cannot create directory ‘Floods/2019-02-20/Sentinel_2_SR’: File exists
Total number of images: 13

Exporting 1/13: 20181223T153619_20181223T153614_T18NUL.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2019-02-20/Sentinel_2_SR/20181223T153619_20181223T153614_T18NUL.tif


Exporting 2/13: 20190102T153619_20190102T153616_T18NUL.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2019-02-20/Sentinel_2_SR/20190102T153619_20190102T153616_T18NUL.tif


Exporting 3/13: 20190112T153619_20190112T153954_T18NUL.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shared dri

'Flood: 2020-06-10 - 2020-07-10'

Floods
mkdir: cannot create directory ‘Floods’: File exists
Floods/2020-06-10
mkdir: cannot create directory ‘Floods/2020-06-10’: File exists
Floods/2020-06-10/Sentinel_2_SR
mkdir: cannot create directory ‘Floods/2020-06-10/Sentinel_2_SR’: File exists
Total number of images: 13

Exporting 1/13: 20200426T153619_20200426T153904_T18NUL.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2020-06-10/Sentinel_2_SR/20200426T153619_20200426T153904_T18NUL.tif


Exporting 2/13: 20200521T153631_20200521T153625_T18NUL.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2020-06-10/Sentinel_2_SR/20200521T153631_20200521T153625_T18NUL.tif


Exporting 3/13: 20200526T153619_20200526T154127_T18NUL.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/Shared dri

##Create Timelapse GIFs of Each Image Collection

In [39]:
%%time

# e.g ['Sentinel_2_SR', 'Landsat_8_T1_SR', etc]. Selects all if empty/None.
desired_collections = ['Sentinel_2_SR', 'Landsat_8_T1_SR', 'Landsat_7_T1_SR', 'Landsat_5_T1_SR', 'MODIS_16D_NDVI'] 

# Set to True to overwrite existing TIFF files.
overwrite = False

# Iterating through each of the test districts.
for district_name, district_geom in test_districts.items():
    display(district_name)
    # Defines base directory to save images to.
    base_dir = f'/content/drive/Shared drives/Colombia SPRINT/Test Districts/{district_name}/Extreme Weather Event Imagery/'
    %cd '{base_dir}'
    
    # Iterating though each disaster in the table.
    for index, disaster in disaster_dates.iterrows():
        disaster_type = disaster['Disaster']
        start_date = disaster['Start_Datetime']
        end_date = disaster['End_Datetime']

        display(f'{disaster_type}: {start_date} - {end_date}')

        # Creates folder for each disaster type. 
        disaster_dir = f'{disaster_type}s'
        print(disaster_dir)

        # Obtains all image collections defined in the request function for 
        # the chosen test district and date range. Looking 2 months either side
        # of the given date range and with maximum 80% cloud cover.
        image_collections = obtain_data(district_geom, 
                                        start_date, 
                                        end_date,
                                        months_either_side=2,
                                        max_cloud_cover=80)
        
        # Filters out unwanted collections if defined above.
        if desired_collections:
            image_collections = {collection: image_collections[collection] \
                                 for collection in desired_collections}

        out_dir = f'{disaster_dir}/{start_date}'
        print(out_dir)

        # Iterating through each image collection.
        for collection_name, collection in image_collections.items():
            # Skips the image collection if it contains no images.
            if collection.size().getInfo() == 0:
                print('No images in collection, skipping.')
                continue

            gif_path = f'{out_dir}/{collection_name}_{start_date}.gif'

            # Skips if GIF already exists, provided overwrite = False.
            if overwrite == False and os.path.isfile(gif_path) == True:
                print(f'{collection_name}_{start_date}.gif already exists, skipping.')
                continue
            
            # Defines GIF parameters depending on the image collection. In this
            # case, specifically for NDVI and RGB images.
            if collection_name == 'MODIS_16D_NDVI':
                # NDVI 
                gif_params = {'bands': bands_dict[collection_name],
                              'dimensions': 1000,
                              'crs': 'EPSG:4326',
                              'framesPerSecond': 1,
                              'min': 0,
                              'max': 8000,
                              'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                                          '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                                          '012E01', '011D01', '011301'],
                              'region': district_geom.geometry().bounds()}
            
            else:
                # RGB
                gif_params = {'bands': bands_dict[collection_name],
                              'dimensions': 1000,
                              'crs': 'EPSG:4326',
                              'framesPerSecond': 1,
                              'min': 0, 
                              'max': 3000,
                              'region': district_geom.geometry().bounds()}

            # Outputs image collection into a timelapse gif.
            emap.download_ee_video(collection, 
                                   gif_params,
                                   gif_path)
            

'Dosquebradas'

/content/drive/Shared drives/Colombia SPRINT/Test Districts/Dosquebradas/Extreme Weather Event Imagery


'Drought: 1998-01-01 - 1999-01-01'

Droughts
Droughts/1998-01-01
No images in collection, skipping.
No images in collection, skipping.
No images in collection, skipping.
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Dosquebradas/Extreme Weather Event Imagery/Droughts/1998-01-01/Landsat_5_T1_SR_1998-01-01.gif
No images in collection, skipping.


'Flood: 1999-01-10 - 1999-05-19'

...

Droughts
Droughts/2018-01-01
Generating URL...
Total request size (39375000 pixels) must be less than or equal to 26214400 pixels.
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Droughts/2018-01-01/Landsat_8_T1_SR_2018-01-01.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Droughts/2018-01-01/Landsat_7_T1_SR_2018-01-01.gif
No images in collection, skipping.
Generating URL...
Total request size (47250000 pixels) must be less than or equal to 26214400 pixels.


'Flood: 2019-02-20 - 2019-02-26'

Floods
Floods/2019-02-20
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2019-02-20/Sentinel_2_SR_2019-02-20.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2019-02-20/Landsat_8_T1_SR_2019-02-20.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2019-02-20/Landsat_7_T1_SR_2019-02-20.gif
No images in collection, skipping.
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2019-02-20/MODIS_16D_NDVI_2019-02-20.gif


'Flood: 2020-06-10 - 2020-07-10'

Floods
Floods/2020-06-10
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2020-06-10/Sentinel_2_SR_2020-06-10.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2020-06-10/Landsat_8_T1_SR_2020-06-10.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2020-06-10/Landsat_7_T1_SR_2020-06-10.gif
No images in collection, skipping.
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Versailles/Extreme Weather Event Imagery/Floods/2020-06-10/MODIS_16D_NDVI_2020-06-10.gif
CPU times: user 9.26 s, sys: 1.18 s, total: 10.4 s
Wall time: 16min 43s
